In [ ]:
https://github.com/Azure/azureml-examples/blob/sdk-preview/sdk/resources/datastores/datastore.ipynb

In [3]:
# import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml import command, Input
from azure.ai.ml.entities import (
    AzureBlobDatastore,
    AzureFileDatastore,
    AzureDataLakeGen1Datastore,
    AzureDataLakeGen2Datastore,
)
from azure.ai.ml.entities import Environment

In [4]:
# Enter details of your AML workspace
subscription_id = "xxxxx"
resource_group = "aml-v2-book"
workspace = "aml2-ws"

In [5]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [24]:
# Create a datastore with account key
blob_datastore1 = AzureBlobDatastore(
    name="blobe_storage",
    description="AML Datastore pointing to a blob storgae.",
    account_name="amlv2sa",
    container_name="datacontainer",   
    credentials={
        "account_key": "XXXxxxXXXxXXXXxxXXXXXxXXXXXxXxxXxXXXxXXXxXXxxxXXxxXXXxXxXXXxxXxxXXXXxxxxxXXxxxxxxXXXxXXX"
    },
)
ml_client.create_or_update(blob_datastore1)

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'blobe_storage', 'description': 'AML Datastore pointing to a blob storgae.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourceGroups/aml-v2-book/providers/Microsoft.MachineLearningServices/workspaces/aml2-ws/datastores/blobe_storage', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f68069a6790>, 'credentials': <azure.ai.ml.entities._datastore.credentials.NoneCredentials object at 0x7f68069d1850>, 'container_name': 'datacontainer', 'account_name': 'amlv2sa', 'endpoint': 'core.windows.net', 'protocol': 'https'})

In [33]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# my_path must point to folder containing MLTable artifact (MLTable file + data
# Supported paths include:
# local: './<path>'
# blob:  'https://<account_name>.blob.core.windows.net/<container_name>/<path>'
# ADLS gen2: 'abfss://<file_system>@<account_name>.dfs.core.windows.net/<path>/'
# Datastore: 'azureml://datastores/<data_store_name>/paths/<path>'

my_path = './my_data/'

my_data = Data(
    path=my_path,
    type=AssetTypes.MLTABLE,
    description="description",
    name="titanic-mltable-sdk",
    version='1'
)

ml_client.data.create_or_update(my_data)

Uploading my_data (0.06 MBs): 100%|██████████| 60607/60607 [00:00<00:00, 976124.07it/s]




Data({'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'titanic-mltable-sdk', 'description': 'description', 'tags': {}, 'properties': {}, 'id': '/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourceGroups/aml-v2-book/providers/Microsoft.MachineLearningServices/workspaces/aml2-ws/data/titanic-mltable-sdk/versions/1', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7f836df7c190>, 'serialize': <msrest.serialization.Serializer object at 0x7f8382418130>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourcegroups/aml-v2-book/workspaces/aml2-ws/datastores/workspaceblobstore/paths/LocalUpload/3d965967cdc59f53b78e4c4a313ce4a1/my_data/', 'referenced_uris': None})

In [37]:
import mltable
tbl = mltable.load(uri="./my_data")
df = tbl.to_pandas_dataframe()
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13,None,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,None,1,2,W./C. 6607,23.45,None,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30,C148,C


In [42]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="env-mltable.yml",
    name="mltable",
    description="Environment created for consuming MLTable.",
)

ml_client.environments.create_or_update(env_docker_conda)

Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'mltable', 'description': 'Environment created for consuming MLTable.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourceGroups/aml-v2-book/providers/Microsoft.MachineLearningServices/workspaces/aml2-ws/environments/mltable/versions/1', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7f836cdb8fd0>, 'serialize': <msrest.serialization.Serializer object at 0x7f836cda9760>, 'version': '1', 'latest_version': None, 'conda_file': OrderedDict([('channels', ['conda-forge']), ('dependencies', ['python=3.8', 'pip=21.2.4', OrderedDict([('pip', ['mltable', 'pandas==1.3.0'])])]), ('name', 'mltable-env')]), 'image': 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04', 'build': None, 'inference_config': None, 'os_type': 'Linux', 'arm_type': 'environment_version', 'conda_file_path': None, 'path': None, 'upload

In [45]:
from azure.ai.ml import command
from azure.ai.ml.entities import Data
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

inputs = {"input_data": Input(type=AssetTypes.MLTABLE, path="./my_data/")}

job = command(
    code=".",  # local path where the code is stored
    command="python read_data.py --input_data ${{inputs.input_data}}",
    inputs=inputs,
    environment=env_docker_conda,
    compute="cpu-cluster",
)

# submit the command
returned_job = ml_client.jobs.create_or_update(job)
# get a URL for the status of the job
returned_job.services["Studio"].endpoint

Uploading code (0.4 MBs): 100%|██████████| 402654/402654 [00:00<00:00, 3345396.72it/s]




'https://ml.azure.com/runs/calm_river_57mrg4b1yn?wsid=/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourcegroups/aml-v2-book/workspaces/aml2-ws&tid=72f988bf-86f1-41af-91ab-2d7cd011db47'

In [47]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

compute_cluster = AmlCompute(
    name="cpu-cluster",
    type="amlcompute",
    size="STANDARD_DS3_v2",
    location="eatus",
    min_instances=0,
    max_instances=2,
    idle_time_before_scale_down=120,
)
ml_client.begin_create_or_update(cluster_basic)

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'basic-example', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/dcfc206a-203b-4c00-a236-bdf576a37896/resourceGroups/aml-v2-book/providers/Microsoft.MachineLearningServices/workspaces/aml2-ws/computes/basic-example', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f836cd477c0>, 'resource_id': None, 'location': 'eastus', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 2, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x7f836cd477f0>, 'tier': 'dedicated'})